### Notebook to test the Rogers (2010) conv/strat classification algorithm being applied to WRF TC output, based on Rosi's F90 algorithm.

James Ruppert  
jruppert@ou.edu  
11/25/22

In [18]:
from netCDF4 import Dataset
import numpy as np
import matplotlib
from matplotlib import ticker
import matplotlib.pyplot as plt
import subprocess
import wrf

#### Directories and settings

In [7]:
storm = 'haiyan'
main = "/ourdisk/hpc/radclouds/auto_archive_notyet/tape_2copies/wrfenkf/"
figdir = "/home/jamesrup/figures/tc/ens/"+storm+'/'

# Time selection
it_read = 37
hr_tag = str(np.char.zfill(str(it_read), 2))

# Simulation and test to read
memb = 'memb_01'
test = 'ctl'

datdir = main+storm+'/'+memb+'/'+test+'/'

#### Read variables

In [20]:
# List of WRF output files
process = subprocess.Popen(['ls '+datdir+'wrfout_d02_*'],shell=True,
    stdout=subprocess.PIPE,universal_newlines=True)
# find desired time
for it in range(it_read): output = process.stdout.readline()

wrffil_path = output.strip() #[3]
wrfnc = Dataset(wrffil_path)
z = wrf.getvar(wrfnc,'z',units='m')
dbz = wrf.getvar(wrfnc,'dbz')
w = wrf.getvar(wrfnc,'wa')
# wrfnc.close()

#### Vertical interpolation

In [21]:
z1 = 400 # m
z2 = 1000 # m
dbz_z = wrf.interplevel(dbz,z,(z1,z2))
w_z = wrf.interplevel(w,z,(z1,z2))

#### Run classification algorithm

In [26]:
dx = wrfnc.DX*1e-3
dy = wrfnc.DY*1e-3

In [27]:
wrfnc.close()

#### Classification algorithm function

In [ ]:
def classify(dbz,w,dx,dy):
    
    nx = dbz.west_east.size
    ny = dbz.north_south.size
    nz = dbz.level.size

    for ik in range(nz):
        